In [1]:
#qsub -t 1:1 embeddings_test.qsub
from miniautoml import train_binary_classifier
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
#import umap.umap_ as umap
from sklearn.metrics import roc_auc_score
import os
import datetime
import pickle

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
lo = [1081, 3503, 4268, 5271, 6229, 6253, 6434, 7210, 7442, 7487, 8655, 8750, 8960, 9289, 9303, 9764, 10307, 10620, 11169, 11206, 11451, 12579, 12675, 13118, 13382, 13801, 14502, 14595, 14787, 15420, 18122, 18779, 20605, 20773, 20887, 21020, 22903, 22904, 22908, 22914, 23571, 23981, 24168, 25730, 26020, 26410, 29327, 29511, 29968, 30107]

In [3]:
#bdb = pd.read_csv("../BindingDB/filtered_bdb_new_subclasses_3_14_25.csv")
bdb = pd.read_csv("../BindingDB/HeavyBDB.csv")
bdb_smiles = bdb["Ligand SMILES"].tolist()
bdb_classes = bdb["Sub_Class"].tolist()
bdb_types = set(bdb_classes)
bdb_class_to_indices = {x:list() for x in bdb_types}
for i, val in enumerate(bdb_classes):
    bdb_class_to_indices[val].append(i)

with open("heavydiverse50.pkl", 'rb') as file:
    diverse50 = pickle.load(file)

print("READINGS EMBEDDINGS")
#embeddings = np.load("../BindingDB/final_bdb_embed.npy").tolist()
embeddings = np.load("../BindingDB/heavy_bdb_embed.npy").tolist()

bdb_total = {x:0 for x in bdb_types}
for x in bdb_classes:
    bdb_total[x] += 1

order = sorted(bdb_total.items(), key=lambda x: -x[1])
classes_of_interest = ["CA2"]
task_id = 0

#newdb = pd.read_csv("../BindingDB/filtered_bdb_new_subclasses_3_14_25.csv")
newdb = pd.read_csv("../BindingDB/HeavyBDB.csv")
mdl = 0
for class_of_interest in classes_of_interest:
    print("STARTING SAMPLING")
    ligands_of_interest = bdb_class_to_indices[class_of_interest]
    #ligands_other = sorted(random.sample(list(set(range(len(bdb_smiles)))-set(ligands_of_interest)), 50)) 
    ligands_other = lo
    #ligands_of_interest = random.sample(ligands_of_interest, 100)
    ligands_of_interest = diverse50[class_of_interest]
    print(ligands_other)
    print(ligands_of_interest)

    X = []
    Y = []
    for i in ligands_of_interest:
        X.append(embeddings[i])
        Y.append(1)
    for i in ligands_other:
        X.append(embeddings[i])
        Y.append(0)
    X = np.array(X)
    Y = np.array(Y)

    print("TRAINING MODELS")
    mdl = train_binary_classifier(X, Y, n_splits=5)
    print(f"{class_of_interest}, self-AUC {mdl.validation_metrics['aucs']}")
'''
    print("TESTING MODELS")
    
    batch_size = 1024
    batch_predictions = []
    for j in range(0, len(embeddings)//batch_size+1):
        batch_predictions.append(mdl.predict(embeddings[batch_size*j:batch_size*(j+1)]))
        print(f"{class_of_interest}: Batch {j}")
    prediction = np.concatenate(batch_predictions)
    current_time = datetime.datetime.now()
    print(current_time.strftime("%D:%H:%M:%S"))
    with open(filename, "a") as file:
        file.write(current_time.strftime("%D:%H:%M:%S") + ": prediction of " + class_of_interest + " complete\n")

    auc_score = roc_auc_score([1 if x == class_of_interest else 0 for x in bdb_classes], prediction)
    print(f"Test AUC: {auc_score}")

    cutoff = len(embeddings) // 100 #len(bdb_class_to_indices["CA12"])
    correct = len([x for x in sorted(range(len(bdb_classes)), key=lambda i: prediction[i])[-cutoff:] if bdb_classes[x] == class_of_interest])
    lift = correct*len(bdb_classes)/(cutoff*len(bdb_class_to_indices[class_of_interest]))
    print(f"Lift: {lift}")

    newdb[class_of_interest] = prediction
    '''


READINGS EMBEDDINGS
STARTING SAMPLING
[1081, 3503, 4268, 5271, 6229, 6253, 6434, 7210, 7442, 7487, 8655, 8750, 8960, 9289, 9303, 9764, 10307, 10620, 11169, 11206, 11451, 12579, 12675, 13118, 13382, 13801, 14502, 14595, 14787, 15420, 18122, 18779, 20605, 20773, 20887, 21020, 22903, 22904, 22908, 22914, 23571, 23981, 24168, 25730, 26020, 26410, 29327, 29511, 29968, 30107]
[5442, 5443, 5444, 5446, 5450, 5451, 5452, 5454, 5455, 5458, 5459, 5460, 5461, 5462, 5464, 5465, 5467, 5468, 5469, 5471, 5472, 5474, 5475, 5476, 5480, 5481, 5482, 5483, 5484, 5485, 5486, 5489, 5490, 5491, 5492, 5493, 5496, 5497, 5499, 5501, 5502, 5503, 5505, 5506, 5507, 5509, 5510, 5511, 5512, 5513]
TRAINING MODELS
CA2, self-AUC [0.64, 0.5900000000000001, 0.55, 0.5800000000000001, 0.61]


'\n    print("TESTING MODELS")\n    \n    batch_size = 1024\n    batch_predictions = []\n    for j in range(0, len(embeddings)//batch_size+1):\n        batch_predictions.append(mdl.predict(embeddings[batch_size*j:batch_size*(j+1)]))\n        print(f"{class_of_interest}: Batch {j}")\n    prediction = np.concatenate(batch_predictions)\n    current_time = datetime.datetime.now()\n    print(current_time.strftime("%D:%H:%M:%S"))\n    with open(filename, "a") as file:\n        file.write(current_time.strftime("%D:%H:%M:%S") + ": prediction of " + class_of_interest + " complete\n")\n\n    auc_score = roc_auc_score([1 if x == class_of_interest else 0 for x in bdb_classes], prediction)\n    print(f"Test AUC: {auc_score}")\n\n    cutoff = len(embeddings) // 100 #len(bdb_class_to_indices["CA12"])\n    correct = len([x for x in sorted(range(len(bdb_classes)), key=lambda i: prediction[i])[-cutoff:] if bdb_classes[x] == class_of_interest])\n    lift = correct*len(bdb_classes)/(cutoff*len(bdb_class_t

In [4]:
dude_embeddings = np.load("../DUD-E/cah2/ca2_dude_embeddings.npy").tolist()

In [5]:
batch_size = 1024
batch_predictions = []
for j in range(0, len(dude_embeddings)//batch_size+1):
    batch_predictions.append(mdl.predict(dude_embeddings[batch_size*j:batch_size*(j+1)]))
    print(f"{class_of_interest}: Batch {j}")
prediction = np.concatenate(batch_predictions)

CA2: Batch 0
CA2: Batch 1
CA2: Batch 2
CA2: Batch 3
CA2: Batch 4
CA2: Batch 5
CA2: Batch 6
CA2: Batch 7
CA2: Batch 8
CA2: Batch 9
CA2: Batch 10
CA2: Batch 11
CA2: Batch 12
CA2: Batch 13
CA2: Batch 14
CA2: Batch 15
CA2: Batch 16
CA2: Batch 17
CA2: Batch 18
CA2: Batch 19
CA2: Batch 20
CA2: Batch 21
CA2: Batch 22
CA2: Batch 23
CA2: Batch 24
CA2: Batch 25
CA2: Batch 26
CA2: Batch 27
CA2: Batch 28
CA2: Batch 29
CA2: Batch 30


In [6]:
np.save("medium_adversarial_ca2_dude_pred.npy", prediction)

In [7]:
len(prediction)

31663